# Importing libraries

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import models, layers

In [3]:
import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

# Creating Dataset

In [4]:

DATADIR = "E:/KJ Somaiya/LY/DL/IA2/color thermal images"

CATEGORIES = ["single cell hotspot", "multicell hotspot", "dust shadow hotspot", "diode fault", "PID effect"]

In [5]:
data = []

def create_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            img_array_bgr = cv2.imread(os.path.join(path,img))
            img_array= cv2.cvtColor(img_array_bgr, cv2.COLOR_BGR2RGB)
            new_array = cv2.resize(img_array, (224, 224))
            data.append([new_array, class_num])
            
create_data()
           
    

In [6]:
len(data)

239

## Shuffling dataset

In [7]:
import random

random.Random(42).shuffle(data)

In [8]:
X = []
y = []

In [9]:
for features, label in data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, 224, 224, 3)

# Data Preprocessing

## Data Normalizing

In [10]:
X = X/255

In [11]:
y = np.array(y)

In [12]:
y = y.reshape(-1,)

## Using SMOTE for balancing dataset

In [13]:
np.bincount(y)

array([55, 52, 39, 43, 50], dtype=int64)

In [14]:
ReX = X.reshape(len(X),-1)

from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 42)

X_smote, y_smote = sm.fit_resample(ReX, y)


In [15]:
X_smote = X_smote.reshape(-1, 224, 224, 3)

In [16]:
X_smote.shape

(275, 224, 224, 3)

In [17]:
np.bincount(y_smote)

array([55, 55, 55, 55, 55], dtype=int64)

## Train-test split

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, stratify=y_smote, test_size = 0.2, random_state = 0)

# Data Augmentation

In [19]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(224, 
                                                              224,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

# CNN

In [20]:
cnn = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.5),
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.5),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(5, activation='sigmoid')
])

cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

cnn.fit(X_train, y_train, epochs=20)

Epoch 1/20
3/7 [===========>..................] - ETA: 5s - loss: 12.3149 - accuracy: 0.2292

KeyboardInterrupt: 

In [52]:
cnn.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 109, 109, 64)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 54, 54, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 186624)           

In [53]:
y_pred_cnn = cnn.predict(X_test)

In [55]:
y_pred_cnn = [np.argmax(element) for element in y_pred_cnn]

In [57]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(y_test, y_pred_cnn))

              precision    recall  f1-score   support

           0       0.62      0.73      0.67        11
           1       0.62      0.45      0.53        11
           2       0.91      0.91      0.91        11
           3       0.90      0.82      0.86        11
           4       0.62      0.73      0.67        11

    accuracy                           0.73        55
   macro avg       0.73      0.73      0.73        55
weighted avg       0.73      0.73      0.73        55



In [58]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, y_pred_cnn))

0.7272727272727273


# Transfer Learning Using MobileNet

In [21]:
X.shape[1:]

(224, 224, 3)

In [22]:
feature_vector = "https://tfhub.dev/google/imagenet/mobilenet_v3_large_075_224/feature_vector/5"
tl_layer = hub.KerasLayer(
    feature_vector, input_shape = X.shape[1:], trainable=False)

OSError: SavedModel file does not exist at: C:\Users\ADMIN\AppData\Local\Temp\tfhub_modules\e9e1d116a3f9f36a97767c2f52452f7256dd57c2\{saved_model.pbtxt|saved_model.pb}

In [78]:
num_classes = 5
#data_augmentation
tl = Sequential([
  #data_augmentation,
  tl_layer,
  layers.Dense(128, activation='relu'),
  #keras.layers.Dropout(0.5),
  layers.Dense(128, activation='relu'),
  layers.Dense(5, activation='sigmoid')
])

tl.compile(loss="sparse_categorical_crossentropy",
             optimizer = "adam",
             metrics = ['accuracy'])
              
tl.fit(X_train, y_train, batch_size = 16, epochs=20, validation_split = 0.1) 

Epoch 1/20
13/13 [==============================] - 7s 399ms/step - loss: 1.2696 - accuracy: 0.4495 - val_loss: 0.8796 - val_accuracy: 0.6364
Epoch 2/20
13/13 [==============================] - 4s 333ms/step - loss: 0.8849 - accuracy: 0.5859 - val_loss: 0.6634 - val_accuracy: 0.7273
Epoch 3/20
13/13 [==============================] - 4s 325ms/step - loss: 0.7626 - accuracy: 0.6717 - val_loss: 0.6521 - val_accuracy: 0.7727
Epoch 4/20
13/13 [==============================] - 4s 326ms/step - loss: 0.6191 - accuracy: 0.7525 - val_loss: 0.5686 - val_accuracy: 0.6818
Epoch 5/20
13/13 [==============================] - 4s 320ms/step - loss: 0.4963 - accuracy: 0.8030 - val_loss: 0.4816 - val_accuracy: 0.7727
Epoch 6/20
13/13 [==============================] - 4s 317ms/step - loss: 0.4505 - accuracy: 0.8333 - val_loss: 0.6693 - val_accuracy: 0.6364
Epoch 7/20
13/13 [==============================] - 4s 315ms/step - loss: 0.3907 - accuracy: 0.8737 - val_loss: 0.7048 - val_accuracy: 0.5909
Epoch 

In [80]:
tl.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2731616   
_________________________________________________________________
dense_16 (Dense)             (None, 128)               163968    
_________________________________________________________________
dense_17 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_18 (Dense)             (None, 5)                 645       
Total params: 2,912,741
Trainable params: 181,125
Non-trainable params: 2,731,616
_________________________________________________________________


In [81]:
y_pred_mnet = tl.predict(X_test)

In [82]:
y_pred_mnet = [np.argmax(element) for element in y_pred_mnet]

In [83]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(y_test, y_pred_mnet))

              precision    recall  f1-score   support

           0       0.78      0.64      0.70        11
           1       0.62      0.73      0.67        11
           2       0.90      0.82      0.86        11
           3       0.83      0.91      0.87        11
           4       0.64      0.64      0.64        11

    accuracy                           0.75        55
   macro avg       0.75      0.75      0.75        55
weighted avg       0.75      0.75      0.75        55



In [84]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, y_pred_mnet))

0.7454545454545455


# Transfer Learning using InceptionNet

In [85]:
feature_vector_in = "https://tfhub.dev/google/imagenet/inception_v1/feature_vector/5"
in_layer = hub.KerasLayer(
    feature_vector, input_shape = X.shape[1:], trainable=False)

In [124]:
num_classes = 5
#data_augmentation
inception = Sequential([
  #data_augmentation,
  in_layer,
  layers.Dense(128, activation='relu'),
  #layers.Dense(128, activation='relu'),
  layers.Dense(5, activation='sigmoid')
])

inception.compile(loss="sparse_categorical_crossentropy",
             optimizer = "adam",
             metrics = ['accuracy'])
              
inception.fit(X_train, y_train, batch_size = 8, epochs=20, validation_split = 0.1) 

Epoch 1/20
25/25 [==============================] - 6s 196ms/step - loss: 1.3401 - accuracy: 0.4747 - val_loss: 0.8914 - val_accuracy: 0.5909
Epoch 2/20
25/25 [==============================] - 4s 169ms/step - loss: 0.8368 - accuracy: 0.6313 - val_loss: 0.8257 - val_accuracy: 0.5455
Epoch 3/20
25/25 [==============================] - 4s 169ms/step - loss: 0.6378 - accuracy: 0.7576 - val_loss: 0.6244 - val_accuracy: 0.6818
Epoch 4/20
25/25 [==============================] - 4s 171ms/step - loss: 0.6866 - accuracy: 0.6818 - val_loss: 0.5462 - val_accuracy: 0.7727
Epoch 5/20
25/25 [==============================] - 4s 171ms/step - loss: 0.5189 - accuracy: 0.7929 - val_loss: 0.6881 - val_accuracy: 0.5455
Epoch 6/20
25/25 [==============================] - 4s 171ms/step - loss: 0.5118 - accuracy: 0.7879 - val_loss: 0.5784 - val_accuracy: 0.7727
Epoch 7/20
25/25 [==============================] - 4s 172ms/step - loss: 0.3890 - accuracy: 0.8535 - val_loss: 0.5109 - val_accuracy: 0.8182
Epoch 

In [125]:
inception.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_2 (KerasLayer)   (None, 1280)              2731616   
_________________________________________________________________
dense_32 (Dense)             (None, 128)               163968    
_________________________________________________________________
dense_33 (Dense)             (None, 5)                 645       
Total params: 2,896,229
Trainable params: 164,613
Non-trainable params: 2,731,616
_________________________________________________________________


In [126]:
y_pred_inet = inception.predict(X_test)

In [127]:
y_pred_inet = [np.argmax(element) for element in y_pred_inet]

In [128]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(y_test, y_pred_inet))

              precision    recall  f1-score   support

           0       0.67      0.73      0.70        11
           1       0.60      0.82      0.69        11
           2       0.90      0.82      0.86        11
           3       0.83      0.91      0.87        11
           4       0.83      0.45      0.59        11

    accuracy                           0.75        55
   macro avg       0.77      0.75      0.74        55
weighted avg       0.77      0.75      0.74        55



In [129]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, y_pred_inet))

0.7454545454545455
